In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

import numpy as np
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, TrainingArguments, Trainer
from torch.utils.data import Dataset
import logging

from datasets import load_dataset

from datasets import load_dataset

# URL of the TSV file
url = "https://raw.githubusercontent.com/MMU-TDMLab/CompLex/master/train/lcp_single_train.tsv"
test_url = "https://raw.githubusercontent.com/MMU-TDMLab/CompLex/refs/heads/master/test-labels/lcp_single_test.tsv"
# Load the TSV file using the csv format
train_data = load_dataset(
    "csv",
    data_files=url,
    delimiter="\t"  # Specify tab-separated values
)

# Inspect the dataset
print(train_data)


val_data = load_dataset(
    "csv",
    data_files=test_url,
    delimiter="\t"  # Specify tab-separated values
)

# Inspect the dataset
print(val_data)

/home/guangyu/anaconda3/envs/MD/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


DatasetDict({
    train: Dataset({
        features: ['id', 'corpus', 'sentence', 'token', 'complexity'],
        num_rows: 7232
    })
})
DatasetDict({
    train: Dataset({
        features: ['id', 'corpus', 'sentence', 'token', 'complexity'],
        num_rows: 887
    })
})


In [2]:
from transformers import AutoTokenizer, AutoModelForMaskedLM, AutoConfig
#from roberta import RobertaForSequenceClassification
from modeling import CLMSequenceClassification


#model_name = "openai-community/gpt2-medium"
model_name = "HuggingFaceTB/SmolLM2-360M"
#config.num_labels=2
tokenizer = AutoTokenizer.from_pretrained(model_name)

tokenizer.padding_side = 'left'
tokenizer.pad_token = tokenizer.eos_token

import torch
import torch.nn as nn
from transformers import AutoModelForSequenceClassification
from transformers.activations import ACT2FN
import random



model = CLMSequenceClassification.from_pretrained(model_name, num_labels=1).to('cuda')
model.config.pad_token_id = tokenizer.eos_token_id
import RoCoFT

RoCoFT.PEFT(model, method='row', rank=3) 

In [3]:
from datasets import DatasetDict

tokenizer.pad_token = tokenizer.eos_token

tokenizer.padding_side = 'left'

def generate_prompt(data_point):
    """
    Generates a prompt for evaluating the humor intensity of an edited headline.
    Args:
        data_point (dict): A dictionary containing 'original', 'edit', and 'meanGrade'.
    Returns:
        str: The formatted prompt as a string.
    """
    return f"""# Sentence: {data_point['sentence']} # Word: {data_point['token']} # Output: The complexity score between word and output is"""  # noqa: E501


# Assuming `dataset` is your DatasetDict
def add_label_column(example):

    example['labels'] = float(example['complexity'])
  
    example['input'] = generate_prompt(example)

    
    return example

train_data = train_data['train'].map(add_label_column)
val_data = val_data['train'].map(add_label_column)

In [4]:
from transformers import AutoTokenizer, DataCollatorWithPadding


tokenizer.padding_side = 'left'


# col_to_delete = ['idx']
col_to_delete = ['id', 'corpus', 'sentence', 'token', 'complexity']

mask_token = tokenizer.mask_token
def preprocessing_function(examples):
   
    return tokenizer(examples['input'], truncation=True, max_length=512)

tokenized_train_data = train_data.map(preprocessing_function, batched=True, remove_columns=col_to_delete)
tokenized_val_data = val_data.map(preprocessing_function, batched=True, remove_columns=col_to_delete)
# llama_tokenized_datasets = llama_tokenized_datasets.rename_column("target", "label")
tokenized_train_data.set_format("torch")
tokenized_val_data.set_format("torch")

# Data collator for padding a batch of examples to the maximum length seen in the batch
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Map: 100%|██████████| 887/887 [00:00<00:00, 24376.54 examples/s]


In [5]:
tokenizer.decode(tokenized_train_data['input_ids'][10])

'# Sentence: Seven days you shall eat unleavened bread, as I commanded you, at the time appointed in the month Abib; for in the month Abib you came out from Egypt. # Word: days # Output: The complexity score between word and output is'

In [6]:
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from scipy.stats import pearsonr, spearmanr

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    # If predictions are logits or have extra dimensions, squeeze
    if predictions.ndim > 1:
        predictions = predictions.squeeze()

    mae = mean_absolute_error(labels, predictions)
    mse = mean_squared_error(labels, predictions)
    rmse = np.sqrt(mse)
    r2 = r2_score(labels, predictions)
    
    # Define an "accuracy" for regression:
    # Example: within some threshold tolerance
    tolerance = 0.1  # you can change this
    acc = np.mean(np.abs(predictions - labels) < tolerance)

    pearson_corr, _ = pearsonr(predictions, labels)
    spearman_corr, _ = spearmanr(predictions, labels)

    return {
        "MAE": mae,
        "MSE": mse,
        "RMSE": rmse,
        "Accuracy": acc,
        "R2": r2,
        "Pearson": pearson_corr,
        "Spearman's Rank": spearman_corr
    }


In [7]:
from transformers import TrainingArguments, Trainer

import time
from transformers import Trainer, TrainingArguments
training_args = TrainingArguments(
    output_dir='dir',
    learning_rate=5e-4,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=64,
    gradient_accumulation_steps= 4,
    num_train_epochs=10,
    weight_decay=0.20,
    evaluation_strategy="steps",
    save_strategy="steps",
    save_total_limit=2,
    save_steps=10000000,
    logging_steps=100,
   
    load_best_model_at_end=True,
    lr_scheduler_type="cosine",  # You can choose from 'linear', 'cosine', 'cosine_with_restarts', 'polynomial', etc.
    warmup_steps=100,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_data,
    eval_dataset=tokenized_val_data,

    data_collator=data_collator,
    compute_metrics=compute_metrics
)

/home/guangyu/anaconda3/envs/MD/lib/python3.10/site-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


[2025-04-27 18:07:12,360] [INFO] [real_accelerator.py:222:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/home/guangyu/anaconda3/envs/MD/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/home/guangyu/anaconda3/envs/MD/compiler_compat/ld: warning: libpthread.so.0, needed by /usr/local/cuda/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/home/guangyu/anaconda3/envs/MD/compiler_compat/ld: warning: libstdc++.so.6, needed by /usr/local/cuda/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/home/guangyu/anaconda3/envs/MD/compiler_compat/ld: warning: libm.so.6, needed by /usr/local/cuda/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/home/guangyu/anaconda3/envs/MD/compiler_compat/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `std::runtime_error::~runtime_error()@GLIBCXX_3.4'
/home/guangyu/anaconda3/envs/MD/compiler_compat/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `__gxx_personality_v0@CXXABI_1.3'
/home/guangyu/anaconda3/envs/MD/compiler_compat/ld: /usr

In [8]:
trainer.train()

Step,Training Loss,Validation Loss,Mae,Mse,Rmse,Accuracy,R2,Pearson,Spearman's rank
100,0.264700,0.031090,0.135405,0.031090,0.176322,0.467869,-0.895972,0.054916,0.063758
200,0.055300,0.016460,0.094073,0.016460,0.128297,0.667418,-0.003798,0.218708,0.247344
300,0.038500,0.018068,0.098251,0.018068,0.134415,0.614431,-0.101831,0.338845,0.355888
400,0.035900,0.011158,0.080680,0.011158,0.105631,0.692221,0.319546,0.569103,0.541388
500,0.026300,0.016052,0.104021,0.016052,0.126698,0.540023,0.021063,0.620964,0.600026
600,0.019400,0.023910,0.134920,0.023910,0.154627,0.343856,-0.458108,0.675713,0.660192
700,0.017800,0.009486,0.074182,0.009486,0.097396,0.732807,0.421505,0.698167,0.666718
800,0.014800,0.008067,0.069539,0.008067,0.089819,0.765502,0.508017,0.722917,0.685395
900,0.012700,0.007712,0.067844,0.007712,0.087820,0.762120,0.529670,0.745725,0.702228
1000,0.010700,0.008835,0.074363,0.008835,0.093996,0.740699,0.461193,0.751360,0.699968


TrainOutput(global_step=4520, training_loss=0.015374723225172643, metrics={'train_runtime': 2727.6408, 'train_samples_per_second': 26.514, 'train_steps_per_second': 1.657, 'total_flos': 38749339478664.0, 'train_loss': 0.015374723225172643, 'epoch': 10.0})